In [6]:
import pickle
import numpy as np
import pandas as pd

location_file_name = '../Data/variation_locations_32k_10.pkl'


In [1]:
# from sklearn.model_selection import train_test_split
import json

# accession_ids = open('../Data/columnIDs.txt', 'r').read().split('\t')

# Shuffle and split the accession IDs
# train_ids, test_ids = train_test_split(accession_ids, test_size=0.2, random_state=41)
# test_ids, val_ids = train_test_split(test_ids, test_size=0.5, random_state=41)

# print(f'Train IDs: {len(train_ids)}')
# print(f'Test IDs: {len(test_ids)}')
# print(f'Validation IDs: {len(val_ids)}')

with open("../Results/splits_test2.json", "r") as f:
        splits = json.load(f)

train_ids = list(map(str, splits.get("train", [])))
val_ids = list(map(str, splits.get("val", [])))
test_ids = list(map(str, splits.get("test", [])))

# Save the IDs as text files
with open('../Data/train_ids.txt', 'w') as f:
    f.write('\t'.join(train_ids))

with open('../Data/test_ids.txt', 'w') as f:
    f.write('\t'.join(test_ids))

with open('../Data/val_ids.txt', 'w') as f:
    f.write('\t'.join(val_ids))


with open('../Data/train_ids_plink.txt', 'w') as f:
    for id in train_ids:
        f.write(f'{id} {id}' + '\n')

with open('../Data/test_ids_plink.txt', 'w') as f:
    for id in test_ids:
        f.write(f'{id} {id}' + '\n')

with open('../Data/val_ids_plink.txt', 'w') as f:
    for id in val_ids:
        f.write(f'{id} {id}' + '\n')


In [25]:
section_locations = pickle.load(open(location_file_name, 'rb'))
corrected_locations = {}

print('Original locations:')
for key in section_locations:
    print(f'Chromosome {key}: {section_locations[key].shape[0]} sections')
# Check locations for each chromosome for overlap and join
# for i in range(1, 6):
#     chromosome_locations = section_locations[i]
#     j = 1
#     while j < len(chromosome_locations):
#         if chromosome_locations[j][0] <= chromosome_locations[j - 1][1]:
#             # print(f'Overlap in positions {chromosome_locations[j - 1][1]} and {chromosome_locations[j][0]} on chromosome {i}')
#             chromosome_locations[j - 1][1] = chromosome_locations[j][1]
#             chromosome_locations = np.delete(chromosome_locations, j, axis=0)
#             j -= 1
#         j += 1

#     corrected_locations[i] = chromosome_locations
    


# print('\nCorrected locations:')
# for key in corrected_locations:
#     print(f'Chromosome {key}: {corrected_locations[key].shape[0]} sections')
        

Original locations:
Chromosome 1: 13 sections
Chromosome 2: 13 sections
Chromosome 3: 14 sections
Chromosome 4: 13 sections
Chromosome 5: 14 sections


In [ ]:
# Generate regions
regions = []
for i in range(1, 6):
    chromosome_locations = section_locations[i]
    for j in range(len(chromosome_locations)):
        regions.append(f'Chr{i}:{chromosome_locations[j][0]}..{chromosome_locations[j][1]}')

joint_regions = ",".join(regions)

In [8]:
df_bioclim = pd.read_csv("../Data/coords_with_bioclim_30s_fixed.csv")
df_soil = pd.read_csv("../Data/coords_with_soil.csv")

# Merge on common columns, e.g., 'FID' and 'IID' if present
merged_df = pd.merge(df_bioclim, df_soil, on=['FID', 'IID', 'LONG', 'LAT'], how='inner')
# Save the merged DataFrame to a new CSV file
merged_df.to_csv("../Data/environmental_variables.csv", index=False)

In [8]:
# Normalize all numeric columns in the merged DataFrame
from sklearn.preprocessing import StandardScaler
import numpy as np

merged_df = pd.read_csv("D:\Masters\ReportData\combined_variables_dataset.csv")

# Create a copy of the merged DataFrame for normalization
normalized_df = merged_df.copy()

# Identify numeric columns (exclude ID columns and coordinates)
id_columns = ['FID', 'IID']  # These should not be normalized
coord_columns = ['LONG', 'LAT']  # Keep coordinates as is for reference

# Get all numeric columns except ID and coordinate columns
numeric_columns = [col for col in normalized_df.columns 
                  if normalized_df[col].dtype in ['int64', 'float64'] 
                  and col not in id_columns + coord_columns]

print(f"Columns to be normalized: {numeric_columns}")
print(f"Number of numeric columns: {len(numeric_columns)}")

# Initialize StandardScaler (mean=0, std=1)
scaler = StandardScaler()

# Normalize the numeric columns
if numeric_columns:
    normalized_df[numeric_columns] = scaler.fit_transform(normalized_df[numeric_columns])
    
    # Save the normalized DataFrame
    normalized_df.to_csv("D:\Masters\ReportData\combined_variables_dataset_normalized.csv", index=False)
    
    # Print some statistics
    print("\nNormalization complete!")
    print(f"Original data shape: {merged_df.shape}")
    print(f"Normalized data shape: {normalized_df.shape}")
    
    # Show sample statistics for verification
    print("\nSample statistics after normalization:")
    print("Mean (should be ~0):")
    print(normalized_df[numeric_columns].mean().round(6))
    print("\nStd (should be ~1):")
    print(normalized_df[numeric_columns].std().round(6))
    
    # Save the scaler for potential inverse transformation later
    import pickle
    with open("../Data/environmental_scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)
    print("\nScaler saved to '../Data/environmental_scaler.pkl' for inverse transformation if needed")
    
else:
    print("No numeric columns found to normalize!")

Columns to be normalized: ['BIO_4_7', 'BIO_10_5', 'BIO_11_6', 'BIO_12_13_16', 'BIO_14_17', 'Soil_Nitrogen_SOC', 'BIO1', 'BIO2', 'BIO3', 'BIO8', 'BIO9', 'BIO15', 'BIO18', 'BIO19', 'clay_mean', 'bdod_mean', 'wv0033_mean', 'phh2o_mean']
Number of numeric columns: 18

Normalization complete!
Original data shape: (1095, 22)
Normalized data shape: (1095, 22)

Sample statistics after normalization:
Mean (should be ~0):
BIO_4_7              0.0
BIO_10_5            -0.0
BIO_11_6            -0.0
BIO_12_13_16         0.0
BIO_14_17            0.0
Soil_Nitrogen_SOC    0.0
BIO1                 0.0
BIO2                 0.0
BIO3                 0.0
BIO8                -0.0
BIO9                -0.0
BIO15               -0.0
BIO18                0.0
BIO19                0.0
clay_mean           -0.0
bdod_mean            0.0
wv0033_mean         -0.0
phh2o_mean           0.0
dtype: float64

Std (should be ~1):
BIO_4_7              1.000457
BIO_10_5             1.000457
BIO_11_6             1.000457
BIO_12_1

In [2]:
def get_regions_per_chromosome(top_snps, window_size):
    lengths = []
    chr_snps = defaultdict(list)
    for region in top_snps[:100]:
        chromosome = region[0]
        position = int(region[1])
        chr_snps[chromosome].append(position)

    regions_per_chr = {}
    for chromosome, positions in chr_snps.items():
        positions.sort()
        merged = []
        for pos in positions:
            start = pos - window_size // 2
            end = pos + window_size // 2
            if not merged:
                merged.append([start, end])
                lengths.append(end - start)
            else:
                prev_start, prev_end = merged[-1]
                if start <= prev_end + window_size // 2 and end - prev_start <= maxlen:
                    merged[-1][1] = max(prev_end, end)
                    lengths[-1] = merged[-1][1] - merged[-1][0]
                else:
                    merged.append([start, end])
                    lengths.append(end - start)
        regions_per_chr[chromosome] = [f'Chr{chromosome}:{start}..{end}' for start, end in merged]

    print(f"Average region length: {np.mean(lengths)}")
    print(f"Max region length: {np.max(lengths)}")
    print(f"Total number of regions: {sum(len(v) for v in regions_per_chr.values())}")
    return regions_per_chr


In [ ]:
# Call the API
import time
import urllib.request
import pickle
from tqdm import tqdm
import os

top_snps = pickle.load(open('../Data/top_snps.pkl', 'rb'))
accession_ids = open('../Data/columnIDs.txt', 'r').read().split('\t')

window_sizes = [512, 1000]

for window_size in window_sizes:
    regions_per_chr = get_regions_per_chromosome(top_snps, window_size)
    os.makedirs(f'../Data/Pseudogenomes/SparSNP_Joint_{window_size}', exist_ok=True)
    for id in tqdm(accession_ids):
        with open(f'../Data/Pseudogenomes/SparSNP_Joint_{window_size}/{id}.fa', 'wb') as f:
            for chromosome, regions in regions_per_chr.items():
                # Process 5 regions at a time
                for i in range(0, len(regions), 4):
                    region_chunk = regions[i:i+4]
                    region_str = ",".join(region_chunk)
                    url = f'https://tools.1001genomes.org/api/v1/pseudogenomes/strains/{id}/regions/{region_str}'
                    sequence_fasta = urllib.request.urlopen(url).read()
                    f.write(sequence_fasta)
                    f.write(b'\n')


Average region length: 1004.4183673469388
Max region length: 1414
Total number of regions: 98


 24%|██▍       | 278/1135 [49:50<2:33:37, 10.76s/it]


KeyboardInterrupt: 

In [ ]:
# TODO: Show where the SNPs are located in the regions

In [1]:
from collections import defaultdict
import numpy as np

maxlen = 131072

chromosome_sizes = {
    1: 30427671,
    2: 19698289,
    3: 23459830,
    4: 18585056,
    5: 26975502,
}

def get_regions(top_snps, window_size, merge_regions=True):
    lengths = []
    maxlen = window_size
    # Group SNPs by chromosome
    chr_snps = defaultdict(list)
    for region in top_snps:
        chromosome = region[0]
        position = int(region[1])
        chr_snps[chromosome].append(position)

    regions = []
    for chromosome, positions in chr_snps.items():
        positions.sort()
        merged = []
        for pos in positions:
            # Center the window on the SNP, but ensure the region is exactly maxlen
            start = pos - maxlen // 2
            end = start + maxlen
            # Ensure start and end are within chromosome size limits
            if start < 0:
                start = 0
                end = maxlen
            if end > chromosome_sizes[chromosome]:
                end = chromosome_sizes[chromosome]
                start = end - maxlen
                if start < 0:
                    start = 0
            
            if not merged or not merge_regions:
                # Either first region or merging is disabled
                merged.append([start, end])
                lengths.append(end - start)
            else:
                prev_start, prev_end = merged[-1]
                # If current start is less than or equal to previous end, merge
                if start <= prev_end:
                    # Merge by extending the previous region to cover the new region
                    new_end = max(prev_end, end)
                    new_start = prev_start
                    
                    # Check if the merged region exceeds maxlen
                    if new_end - new_start > maxlen:
                        # Need to choose which SNPs to prioritize
                        # Option 1: Keep the region at maxlen starting from the first SNP
                        # This might exclude the current SNP
                        if new_start + maxlen < pos + maxlen // 2:
                            # Current SNP would be too far from center, start new region
                            merged.append([start, end])
                            lengths.append(end - start)
                        else:
                            # Trim to maxlen, keeping first SNP well covered
                            merged[-1][1] = new_start + maxlen
                            lengths[-1] = maxlen
                    else:
                        # Merged region is within maxlen, update it
                        merged[-1][1] = new_end
                        lengths[-1] = new_end - new_start
                else:
                    merged.append([start, end])
                    lengths.append(end - start)
        
        # Add merged regions to the list, ensuring each is exactly maxlen
        for start, end in merged:
            regions.append(f'Chr{chromosome}:{int(start)}..{int(end)}')

    joint_regions = ",".join(regions)
    merge_status = "with merging" if merge_regions else "without merging"
    print(f"Regions created {merge_status}")
    print(f"Average region length: {np.mean(lengths)}")
    print(f"Max region length: {np.max(lengths)}")
    print(f"Total number of regions: {len(regions)}")
    return joint_regions

In [2]:
# Call the API
import time
import pickle
import urllib.request
from tqdm import tqdm
import os
import ssl

# Create an SSL context that doesn't verify certificates (use with caution)
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

top_snps = pickle.load(open('../Data/Pseudogenomes/top_snps_200.pkl', 'rb'))
accession_ids = open('../Data/columnIDs.txt', 'r').read().split('\t')

window_sizes = [512, 1000]

for window_size in window_sizes:
    # Set merge_regions=False to prevent regions from merging
    joint_regions = get_regions(top_snps, window_size, merge_regions=False)
    os.makedirs(f'../Data/Pseudogenomes/SparSNP_FixedLen_{window_size}', exist_ok=True)
    for id in tqdm(accession_ids):
        url = f'https://tools.1001genomes.org/api/v1/pseudogenomes/strains/{id}/regions/{joint_regions}'
        try:
            # Use the SSL context that doesn't verify certificates
            sequence_fasta = urllib.request.urlopen(url, context=ssl_context).read()
            with open(f'../Data/Pseudogenomes/SparSNP_FixedLen_200_{window_size}/{id}.fa', 'wb') as f:
                f.write(sequence_fasta)
        except Exception as e:
            print(f"Error downloading {id}: {e}")
            continue
    
    #API is limited to 2 requests per second, be safe
    # time.sleep(0.2)

Regions created without merging
Average region length: 512.0
Max region length: 512
Total number of regions: 200


  0%|          | 3/1135 [00:00<02:02,  9.24it/s]

Error downloading 88: HTTP Error 403: Forbidden
Error downloading 108: HTTP Error 403: Forbidden
Error downloading 139: HTTP Error 403: Forbidden


  0%|          | 5/1135 [00:00<01:42, 11.04it/s]

Error downloading 159: HTTP Error 403: Forbidden
Error downloading 265: HTTP Error 403: Forbidden
Error downloading 350: HTTP Error 403: Forbidden


...

  2%|▏         | 27/1135 [00:02<01:41, 10.89it/s]

Error downloading 687: HTTP Error 403: Forbidden
Error downloading 728: HTTP Error 403: Forbidden
Error downloading 742: HTTP Error 403: Forbidden


  3%|▎         | 31/1135 [00:02<01:36, 11.46it/s]

Error downloading 763: HTTP Error 403: Forbidden
Error downloading 765: HTTP Error 403: Forbidden
Error downloading 766: HTTP Error 403: Forbidden


KeyboardInterrupt: 

In [12]:
from collections import defaultdict
import numpy as np

maxlen = 1_000_000

chromosome_sizes = {
    1: 30427671,
    2: 19698289,
    3: 23459830,
    4: 18585056,
    5: 26975502,
}


def get_regions_batched(top_snps, window_size, max_snps_per_batch=100, merge_regions=True):
    """
    Create regions from SNPs, splitting into batches if there are too many SNPs.
    Returns a list of region strings, one per batch.
    """
    lengths = []
    maxlen = window_size
    
    # Limit the number of SNPs if there are too many
    if len(top_snps) > max_snps_per_batch:
        print(f"Too many SNPs ({len(top_snps)}), splitting into batches of {max_snps_per_batch}")
        
        # Split SNPs into batches
        snp_batches = []
        for i in range(0, len(top_snps), max_snps_per_batch):
            snp_batches.append(top_snps[i:i + max_snps_per_batch])
    else:
        snp_batches = [top_snps]
    
    all_region_strings = []
    
    for batch_idx, snp_batch in enumerate(snp_batches):
        print(f"Processing batch {batch_idx + 1}/{len(snp_batches)} with {len(snp_batch)} SNPs")
        
        # Group SNPs by chromosome for this batch
        chr_snps = defaultdict(list)
        for region in snp_batch:
            chromosome = region[0]
            position = int(region[1])
            chr_snps[chromosome].append(position)

        regions = []
        batch_lengths = []
        
        for chromosome, positions in chr_snps.items():
            positions.sort()
            used_positions = []
            
            for pos in positions:
                # Check if this SNP would fall within the window of any previously used SNP
                skip_snp = False
                for used_pos in used_positions:
                    # Calculate the distance between SNPs
                    if abs(pos - used_pos) < maxlen:
                        skip_snp = True
                        break
                
                if skip_snp:
                    continue
                
                # Center the window on the SNP, but ensure the region is exactly maxlen
                start = pos - maxlen // 2
                end = start + maxlen
                # Ensure start and end are within chromosome size limits
                if start < 0:
                    start = 0
                    end = maxlen
                if end > chromosome_sizes[chromosome]:
                    end = chromosome_sizes[chromosome]
                    start = end - maxlen
                    if start < 0:
                        start = 0
                
                # Add this region and mark the SNP position as used
                regions.append(f'Chr{chromosome}:{int(start)}..{int(end)}')
                batch_lengths.append(end - start)
                used_positions.append(pos)

        joint_regions = ",".join(regions)
        all_region_strings.append(joint_regions)
        lengths.extend(batch_lengths)
        
        print(f"Batch {batch_idx + 1} regions created without overlapping windows")
        print(f"Batch {batch_idx + 1} regions: {len(regions)}")

    print(f"Total average region length: {np.mean(lengths)}")
    print(f"Total max region length: {np.max(lengths)}")
    print(f"Total number of regions across all batches: {len(lengths)}")
    print(f"Number of batches: {len(all_region_strings)}")
    
    return all_region_strings

In [13]:
import time
import pickle
import urllib.request
from tqdm import tqdm
import os
import ssl

# Create an SSL context that doesn't verify certificates (use with caution)
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

top_snps = pickle.load(open('../Data/Pseudogenomes/top_snps_500.pkl', 'rb'))

# Sort SNPs by chromosome first, then by position
top_snps_sorted = sorted(top_snps, key=lambda x: (x[0], int(x[1])))
print(f"Sorted {len(top_snps_sorted)} SNPs by chromosome and position")

accession_ids = open('../Data/columnIDs.txt', 'r').read().split('\t')

window_sizes = [512]
max_snps_per_request = 100  # Adjust this based on what works for your API

for window_size in window_sizes:
    # Get batched region strings
    region_batches = get_regions_batched(top_snps_sorted, window_size, max_snps_per_request, merge_regions=False)
    
    os.makedirs(f'../Data/Pseudogenomes/SparSNP_FixedLen_500_{window_size}', exist_ok=True)
    
    for id in tqdm(accession_ids, desc=f"Processing accessions for window size {window_size}"):
        all_sequences = []
        
        # Process each batch of regions
        for batch_idx, joint_regions in enumerate(region_batches):
            url = f'https://tools.1001genomes.org/api/v1/pseudogenomes/strains/{id}/regions/{joint_regions}'
            try:
                # Use the SSL context that doesn't verify certificates
                sequence_fasta = urllib.request.urlopen(url, context=ssl_context).read()
                all_sequences.append(sequence_fasta)
                
                # Be nice to the API - small delay between batch requests
                if len(region_batches) > 1:
                    time.sleep(0.1)
                    
            except Exception as e:
                print(f"Error downloading batch {batch_idx + 1} for {id}: {e}")
                continue
        
        # Combine all sequences and write to file
        if all_sequences:
            with open(f'../Data/Pseudogenomes/SparSNP_FixedLen_500_{window_size}/{id}.fa', 'wb') as f:
                for seq in all_sequences:
                    f.write(seq)
                    # Add newline between batches if needed
                    if not seq.endswith(b'\n'):
                        f.write(b'\n')
        
        # API rate limiting
        time.sleep(0.1)

Sorted 500 SNPs by chromosome and position
Too many SNPs (500), splitting into batches of 100
Processing batch 1/5 with 100 SNPs
Batch 1 regions created without overlapping windows
Batch 1 regions: 97
Processing batch 2/5 with 100 SNPs
Batch 2 regions created without overlapping windows
Batch 2 regions: 99
Processing batch 3/5 with 100 SNPs
Batch 3 regions created without overlapping windows
Batch 3 regions: 100
Processing batch 4/5 with 100 SNPs
Batch 4 regions created without overlapping windows
Batch 4 regions: 99
Processing batch 5/5 with 100 SNPs
Batch 5 regions created without overlapping windows
Batch 5 regions: 97
Total average region length: 512.0
Total max region length: 512
Total number of regions across all batches: 492
Number of batches: 5


Processing accessions for window size 512: 100%|██████████| 1135/1135 [30:55<00:00,  1.63s/it]


In [ ]:
# Alternative: Using requests library (more robust)
# First install: pip install requests
# import requests

# for id in tqdm(accession_ids):
#     url = f'https://tools.1001genomes.org/api/v1/pseudogenomes/strains/{id}/regions/{joint_regions}'
#     try:
#         response = requests.get(url, verify=False)  # verify=False disables SSL verification
#         response.raise_for_status()
#         with open(f'../Data/Pseudogenomes/SparSNP_FixedLen_{window_size}/{id}.fa', 'wb') as f:
#             f.write(response.content)
#     except Exception as e:
#         print(f"Error downloading {id}: {e}")
#         continue

In [ ]:
# Call the SoilGrids API
import time
import pickle
import os
import urllib.request
from tqdm import tqdm

coverage = "phh2o_5-15cm_uncertainty"

os.makedirs(f'../Data/SoilGrids/{coverage}', exist_ok=True)

accession_ids = open('../Data/columnIDs.txt', 'r').read().split('\t')
sample_map_array = np.ndarray((0, 2))

areas = pickle.load(open('../Data/sample_areas.pkl', 'rb'))

for region in tqdm(areas):

    min_long = region["min_long"]
    max_long = region["max_long"]
    min_lat = region["min_lat"]
    max_lat = region["max_lat"]

    url = f'https://maps.isric.org/mapserv?map=/map/{coverage.split("_")[0]}.map&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID={coverage}&FORMAT=image/tiff&SUBSET=long({min_long:.4f},{max_long:.4f})&SUBSET=lat({min_lat:.4f},{max_lat:.4f})&SUBSETTINGCRS=http://www.opengis.net/def/crs/EPSG/0/4326&OUTPUTCRS=http://www.opengis.net/def/crs/EPSG/0/4326'
    tif_file = urllib.request.urlopen(url).read()
    file_name = f'../Data/SoilGrids/{coverage}/{coverage}_{int(min_long)}_{int(max_long)}_{int(min_lat)}_{int(max_lat)}.tif'
    with open(file_name, 'wb') as f:
        f.write(tif_file)
    for key in region["ids"]:
        sample_map_array = np.vstack((sample_map_array, [key, file_name]))

pickle.dump(sample_map_array, open(f'../Data/SoilGrids/{coverage}/sample_image_map.pkl', 'wb'))

100%|██████████| 86/86 [01:40<00:00,  1.17s/it]


In [24]:

bioclim_file = "../Data/coords_with_bioclim_30s.csv"
bioclim = pd.read_csv(bioclim_file)

filename = "../Data/bioclim_6.fam"

with open(filename, "w+") as f:
    for index, row in bioclim.iterrows():

        f.write(f"{row['FID']:.0f} {row['IID']:.0f} 0 0 0 {row['BIO6']}\n")

In [12]:
missing_file = "../Data/missing_data.txt"
filename = "../Data/plink.fam"
with open(filename, 'r') as f:
    with open(missing_file, 'w') as m:
        lines = f.readlines()
        for line in lines:
            if line.strip().split(" ")[5] == "-3.4e+38":
                m.write(f"{line.split(' ')[0]} {line.split(' ')[1]}\n")

In [21]:
fam_file = "../Data/plink_bio6_train.fam"
test_file = "../Data/plink_bio6_test.fam"

csv = pd.read_csv(fam_file, sep=" ", header=None)
csv_test = pd.read_csv(test_file, sep=" ", header=None)

mean = csv[5].mean()
std = csv[5].std()

csv[5] = (csv[5] - mean) / std
csv_test[5] = (csv_test[5] - mean) / std

csv.to_csv(fam_file, sep=" ", header=False, index=False)
csv_test.to_csv(test_file, sep=" ", header=False, index=False)



In [22]:
print(mean, std)

-3.821640091116173 5.467024484498266


In [14]:
bim_file = "../Data/plink.bim"
out_file = "../Data/renamed.bim"

with open(bim_file, 'r') as f:
    with open(out_file, 'w') as o:
        lines = f.readlines()
        for line in lines:
            split = line.split('\t')
            split[1] = f"{split[0]}_{split[3]}"
            o.write('\t'.join(split))

In [ ]:
# concat embeddings
folder_name = '../Data/Embeddings/Variance/'
import pickle


embeddings = {}
for id in accession_ids:
    embed = pickle.load(open(f'{folder_name}{id}.embed', 'rb'))
    embeddings[id] = embed

In [4]:
pickle.dump(embeddings, open(f'../Data/Embeddings/Variance-Embeddings.pkl', 'wb'))